# Survival Function Drawing

Notebook to explore implementing a better workflow for drawing the survival functions. Motivated by the fact that it seems like, when $\beta$ is chosen to be very small as in Mammen and Polonik, there can actually be no points that are contained within the region over which you will take the sup. This is a problem, and so we will be exploring alternative ways to draw the survival function, specifically thinking about how we might draw it only in the actual region of interest and then compute the sup difference. Here, I will be experimenting with different ways to draw it and comparing how long it takes relative to the old methods to guarantee that we aren't sacrificing too much in terms of computational power..

Jimmy Butler, December 2024

In [4]:
library(mvtnorm)
library(mnormt)
library(ggplot2)
library(tidyr)

source('~/isolines_uq/scripts/R/auxiliary_scripts/distributionIsolines.R')

## Method 1: Polar Coordinate Method with Radial Grid Points

In [7]:
lbs <- c(-2,-2)
ubs <- c(10,10)
p <- 0.01
beta <- 0.001
n_iso_pts <- 100

lower <- drawBivtIsoline(numCoords=n_iso_pts, gridUbs=c(10,10), gridLbs=c(-2,-2), prob=p+beta)
upper <- drawBivtIsoline(numCoords=n_iso_pts, gridUbs=c(10,10), gridLbs=c(-2,-2), prob=p-beta)

In [26]:
drawSupRegion <- function(n_boundary_coords, n_interior_coords, gridLbs, gridUbs, p, beta) {
    
    pmvtFixedCoord <- function(radius, angle, sigma, df, prob, xCenter, yCenter) {
        xCoord <- xCenter + radius*cos(angle)
        yCoord <- yCenter + radius*sin(angle)
        exceedanceProb <- pmvt(lower=c(xCoord, yCoord), upper=c(Inf, Inf), df=df, sigma=sigma)
        return(exceedanceProb - prob)
    }
    
    radii <- rep(NA, n_boundary_coords*n_interior_coords)
    full_angles <- rep(NA, n_boundary_coords*n_interior_coords)
    angles <- seq(0, pi/2, length.out=n_boundary_coords)
    maxRad <- sqrt(sum((gridUbs-gridLbs)**2))

    # for each angle in first quadrant, find radius that gives a point with desired exceedance probability
    # by finding roots of pmvtFixedCoord given the angle
    for (i in 1:n_boundary_coords) {
        angle <- angles[i]
        lower_radius <- uniroot(pmvtFixedCoord, interval=c(0, maxRad), angle=angle,
                             sigma=matrix(c(1, 0.7, 0.7, 1), nrow=2), df=4, prob=p+beta, xCenter=gridLbs[1], yCenter=gridLbs[2])$root
        upper_radius <- uniroot(pmvtFixedCoord, interval=c(0, maxRad), angle=angle,
                             sigma=matrix(c(1, 0.7, 0.7, 1), nrow=2), df=4, prob=p-beta, xCenter=gridLbs[1], yCenter=gridLbs[2])$root
        
        radii_per_angle <- seq(lower_radius, upper_radius, length.out=n_interior_coords)
        angles_per_angle <- rep(angle, length.out=n_interior_coords)
        
        radii[((i-1)*n_interior_coords + 1):(i*n_interior_coords)] <- radii_per_angle
        full_angles[((i-1)*n_interior_coords + 1):(i*n_interior_coords)] <- angles_per_angle
        
        
    }

    # convert back to cartesian coordinates
    xs <- radii*cos(full_angles) + gridLbs[1]
    ys <- radii*sin(full_angles) + gridLbs[2]
    
    return(cbind(xs, ys))
    
    
}

In [71]:
sample_dat <- data.frame(rmvt(10000, sigma = matrix(c(1, 0.7, 0.7, 1), nrow = 2), df = 4))

In [86]:
region <- drawSupRegion(100, 5, c(-2,-2), c(10,10), 0.01, 0.001)

In [87]:
true_survfunc <- pmt(x=as.matrix(region), mean=rep(0,2), S=matrix(c(1,0.7,0.7,1), 2,2), df=4)

In [88]:
empSurv <- function(pt, dat) {
    point <- as.numeric(pt)
    return(mean((point[1] < dat[,1]) & (point[2] < dat[,2])))
}

In [91]:
start <- proc.time()
res <- apply(region, 1, empSurv, dat=sample_dat)
proc.time() - start

   user  system elapsed 
  0.073   0.000   0.073 